<a href="https://www.kaggle.com/code/vijays140291/oscillation-prediction?scriptVersionId=194873254" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [121]:
#importing library
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [122]:
file1='/kaggle/input/oscillilation1212/Valeur_Flottante_QV1.csv'
file2='/kaggle/input/oscillilation1212/Valeur_Flottante_QV2.csv'
file3='/kaggle/input/oscillilation1212/Valeur_Flottante_QV3.csv'
file4='/kaggle/input/oscillilation1212/Valeur_Flottante_QV4.csv'
file5='/kaggle/input/oscillilation1212/Valeur_Flottante_QV5.csv'
file6='/kaggle/input/oscillilation1212/Valeur_Flottante_QVP.csv'




df=pd.read_csv(file6)
df

,Cycle Cumulé,Valeur suivante,Valeur flottante
0,0,0,0
1,0,-2,-2
2,-2,-8,-10
3,-10,-10,-18
4,-20,6,-4
...,...,...,...
2526,13,-12,-3
2527,1,-7,-19
2528,-6,14,7
2529,8,-2,12


In [123]:
y=df['Valeur suivante']
x=df.drop('Valeur suivante',axis=1)

# Before Adding Features

In [124]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.9,random_state=122)
scaler=StandardScaler()
scaler.fit(x_train)
x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns,index=x_train.index)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns,index=x_test.index)

# Before hypertuning

In [125]:
model=RandomForestRegressor()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.3467721014920704

# Result

# After hypertuning

In [126]:
model = RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=100)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.4412763631946366

In [127]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor



model = RandomForestRegressor()

param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20],    # Maximum depth of each tree
    'min_samples_split': [2, 5, 10]  # Minimum samples required to split a node
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')


grid_search.fit(x_train, y_train)


best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


best_model = grid_search.best_estimator_
test_score = best_model.score(x_test, y_test)
print(f"Test Score (R^2): {test_score:.4f}")


Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 200}
Test Score (R^2): 0.4383


# After adding new Features

In [128]:
df_new=df.copy()
df_new['Cycle greater or less']=df_new['Cycle Cumulé'].apply(lambda x:1 if x>0 else 0)
df_new['Valeur greater or less']=df_new['Valeur flottante'].apply(lambda x:1 if x>0 else 0)
df_new

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less
0,0,0,0,0,0
1,0,-2,-2,0,0
2,-2,-8,-10,0,0
3,-10,-10,-18,0,0
4,-20,6,-4,0,0
...,...,...,...,...,...
2526,13,-12,-3,1,0
2527,1,-7,-19,1,0
2528,-6,14,7,0,1
2529,8,-2,12,1,1


In [129]:

df_new['Cycle_Moving Average'] = df_new['Cycle Cumulé'].rolling(window=2).mean()

df_new['Cycle_Cumulative Deviation'] = (df_new['Cycle Cumulé'] - df_new['Cycle Cumulé'].mean()).cumsum()
df_new['Valeur_Moving Average'] = df_new['Valeur flottante'].rolling(window=2).mean()

df_new['Valeur_Cumulative Deviation'] = (df_new['Valeur flottante'] - df_new['Valeur flottante'].mean()).cumsum()


In [130]:
df_new

,Cycle Cumulé,Valeur suivante,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation
0,0,0,0,0,0,NaN,3.469775e+00,NaN,1.303832e-02
1,0,-2,-2,0,0,0.0,6.939550e+00,-1.0,-1.973923e+00
2,-2,-8,-10,0,0,-1.0,8.409324e+00,-6.0,-1.196089e+01
3,-10,-10,-18,0,0,-6.0,1.879099e+00,-14.0,-2.994785e+01
4,-20,6,-4,0,0,-15.0,-1.465113e+01,-11.0,-3.393481e+01
...,...,...,...,...,...,...,...,...,...
2526,13,-12,-3,1,0,8.5,-2.287910e+01,2.5,-3.052153e+00
2527,1,-7,-19,1,0,7.0,-1.840932e+01,-11.0,-2.203911e+01
2528,-6,14,7,0,1,-2.5,-2.093955e+01,-6.0,-1.502608e+01
2529,8,-2,12,1,1,1.0,-9.469775e+00,9.5,-3.013038e+00


In [131]:
df_new.columns

Index(['Cycle Cumulé', 'Valeur suivante', 'Valeur flottante',
       'Cycle greater or less', 'Valeur greater or less',
       'Cycle_Moving Average', 'Cycle_Cumulative Deviation',
       'Valeur_Moving Average', 'Valeur_Cumulative Deviation'],
      dtype='object')

In [132]:
df_new['Cycle_Moving Average']=df_new['Cycle_Moving Average'].fillna(df_new['Cycle_Moving Average'].mean())

In [133]:
df_new['Valeur_Moving Average']=df_new['Valeur_Moving Average'].fillna(df_new['Valeur_Moving Average'].mean())

In [134]:
df_train=df_new.loc[1:2500]
df_test=df_new.loc[2500:]

In [135]:

column_to_exclude = 'Valeur suivante'

# Separate the column you don't want to scale
column_excluded = df_train[[column_to_exclude]]

# Select the remaining data
data_to_scale = df_train.drop(columns=[column_to_exclude])

# Apply StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_to_scale)

# Convert scaled data back to DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns, index=df_train.index)

# Combine the scaled data with the excluded column
df_train = pd.concat([scaled_df, column_excluded], axis=1)

print(df_train)


      Cycle Cumulé  Valeur flottante  Cycle greater or less  \
1         0.252643         -0.140825              -0.770306   
2         0.106876         -0.710735              -0.770306   
3        -0.476191         -1.280645              -0.770306   
4        -1.205024         -0.283302              -0.770306   
5        -0.767724         -0.781974              -0.770306   
...            ...               ...                    ...   
2496     -0.330424         -0.212064              -0.770306   
2497     -0.767724          0.429085              -0.770306   
2498      0.106876         -0.497019              -0.770306   
2499     -1.277907         -0.853212              -0.770306   
2500     -0.767724          0.357847              -0.770306   

      Valeur greater or less  Cycle_Moving Average  \
1                  -0.956927              0.294415   
2                  -0.956927              0.209413   
3                  -0.956927             -0.215601   
4                  -0.95692

In [136]:
df_train

,Cycle Cumulé,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation,Valeur suivante
1,0.252643,-0.140825,-0.770306,-0.956927,0.294415,2.546449,-0.099028,1.456379,-2
2,0.106876,-0.710735,-0.770306,-0.956927,0.209413,2.548206,-0.606449,0.771383,-8
3,-0.476191,-1.280645,-0.770306,-0.956927,-0.215601,2.540397,-1.418322,-0.462325,-10
4,-1.205024,-0.283302,-0.770306,-0.956927,-0.980625,2.520629,-1.113870,-0.735787,6
5,-0.767724,-0.781974,-0.770306,-0.956927,-1.150630,2.508037,-0.758675,-1.489372,-17
...,...,...,...,...,...,...,...,...,...
2496,-0.330424,-0.212064,-0.770306,-0.956927,-0.513110,2.596780,-0.200512,0.395349,-6
2497,-0.767724,0.429085,-0.770306,1.045012,-0.640614,2.584187,0.154682,0.807778,12
2498,0.106876,-0.497019,-0.770306,-0.956927,-0.385606,2.585945,-0.048286,0.328549,-19
2499,-1.277907,-0.853212,-0.770306,-0.956927,-0.683115,2.564981,-0.961643,-0.493625,7


In [137]:

column_to_exclude = 'Valeur suivante'

# Separate the column you don't want to scale
column_excluded = df_test[[column_to_exclude]]

# Select the remaining data
data_to_scale = df_test.drop(columns=[column_to_exclude])

# Apply StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_to_scale)

# Convert scaled data back to DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns, index=df_test.index)

# Combine the scaled data with the excluded column
df_test = pd.concat([scaled_df, column_excluded], axis=1)
df_test


,Cycle Cumulé,Valeur flottante,Cycle greater or less,Valeur greater or less,Cycle_Moving Average,Cycle_Cumulative Deviation,Valeur_Moving Average,Valeur_Cumulative Deviation,Valeur suivante
2500,-1.153935,0.399192,-0.639602,0.794719,-1.987191,2.354400,-0.663778,-1.962339,-2
2501,-1.389277,1.092189,-0.639602,0.794719,-1.601208,1.637990,1.221150,-0.540755,14
2502,0.258117,0.003194,-0.639602,0.794719,-0.674848,1.722024,0.906995,-0.420876,-13
2503,-1.271606,-1.481801,-0.639602,-1.258306,-0.597651,1.062789,-1.135010,-2.076047,-1
2504,-1.389277,1.587187,-0.639602,0.794719,-1.678404,0.346379,0.121608,-0.062780,18
2505,0.728801,0.102193,1.563472,0.794719,-0.366061,0.659111,1.378227,0.175437,-16
2506,-1.153935,-0.392805,-0.639602,-1.258306,-0.211668,0.057051,-0.192546,-0.178031,13
2507,0.375788,1.092189,-0.639602,0.794719,-0.443258,0.198259,0.592840,1.243553,-1
2508,0.258117,-2.174798,-0.639602,-1.258306,0.483102,0.282292,-0.820856,-1.239975,-20
2509,-2.095303,0.399192,-0.639602,0.794719,-1.138028,-0.777164,-1.370626,-0.646749,25


In [138]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Example data (5 columns)
data = np.array(df_train)

# Prepare sequences (windowed sequences of input data)
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Use all columns in the sequence
        y.append(data[i+seq_length, target_column])  # Target is the next value in the target column
    return np.array(X), np.array(y)

seq_length = 2
target_column = 8  # We want to predict the next value of the 3rd column
X, y = create_sequences(data, seq_length, target_column)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=200, verbose=0)

# Predict the next value for a new sequence
test_sequence = np.array(df_test.loc[2529:2530])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 2rd column: {predicted_value[0][0]}")


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Predicted next value for the 2rd column: 2.286109447479248


In [139]:
# Predict the next value for a new sequence
test_sequence = np.array(df_test.loc[2505:2506])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 2rd column: {predicted_value[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted next value for the 2rd column: -19.98322296142578


In [140]:
scaler=StandardScaler()
scaler.fit(x_train_new)
x_train_new=pd.DataFrame(scaler.transform(x_train_new),columns=x_train_new.columns,index=x_train_new.index)
x_test_new=pd.DataFrame(scaler.transform(x_test_new),columns=x_test_new.columns,index=x_test_new.index)
final_test_set=pd.DataFrame(scaler.transform(final_test_set),columns=final_test_set.columns,index=final_test_set.index)

NameError: name 'x_train_new' is not defined

In [ ]:
final_test_set

# Result

In [ ]:
model_new = RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=100)
model_new.fit(x_train_new,y_train_new)
model_new.score(x_test_new,y_test_new)

In [ ]:
y_test_new

In [ ]:
y_pred=model_new.predict(x_test_new)

# Root Mean Square Error

In [ ]:
np.sqrt(np.mean((y_pred-y_test_new)**2))

# XGBOOST REGRESSOR

In [ ]:
y_pred = model.predict(x_test_new)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test_new, y_pred, squared=False)
print(f'RMSE: {rmse}')


In [ ]:
model.score(x_test_new,y_test_new)

In [ ]:
model.predict(final_test_set)

# Actual Value is 15 and predicted value is 16.43

In [ ]:
model.predict(x_test_new)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(20, 6))

# Plot the actual values
plt.plot(y_test_new.reset_index(drop=True), label='Actual Values', color='blue', linestyle='-', marker='o')

# Plot the predicted values
plt.plot(y_pred, label='Predicted Values', color='red', linestyle='--', marker='x')

# Add labels and title
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.title('Actual vs Predicted Values')

# Show legend to distinguish between actual and predicted lines
plt.legend()

# Display the plot
plt.show()


In [ ]:
x_train_new.shape

# NEURAL NETWORK

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Example data (5 columns)
data = np.array([[0.1, 0.2, 0.3, 0.4, 0.5],
                 [0.6, 0.7, 0.8, 0.9, 1.0],
                 [1.1, 1.2, 1.3, 1.4, 1.5],
                 [1.6, 1.7, 1.8, 1.9, 2.0]])

# Prepare sequences (windowed sequences of input data)
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Use all columns in the sequence
        y.append(data[i+seq_length, target_column])  # Target is the next value in the target column
    return np.array(X), np.array(y)

seq_length = 2
target_column = 2  # We want to predict the next value of the 3rd column
X, y = create_sequences(data, seq_length, target_column)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=200, verbose=0)

# Predict the next value for a new sequence
test_sequence = np.array([[1.1, 1.2, 1.3, 1.4, 1.5],
                          [1.6, 1.7, 1.8, 1.9, 2.0]])
test_sequence = test_sequence.reshape((1, seq_length, test_sequence.shape[1]))

predicted_value = model.predict(test_sequence)
print(f"Predicted next value for the 3rd column: {predicted_value[0][0]}")


In [ ]:
tf_model.evaluate(x_test_new,y_test_new)

In [ ]:
y_pred = model.predict(x_test_new)
